# Задача 27 (Самая сложная программа в ЕГЭ)

In [1]:
import pandas as pd
answers = pd.read_csv('answers.csv', encoding='windows-1251')['27'].to_list()
answers = [None] + answers

solutions = [None] * (len(answers)+1) # solutions[i] - решение i-й задачи. solutions[0] не используется.

In [2]:
answers[13]

'30 360137507'

In [3]:
test_env = True # если True, то тесты будут запущены

In [4]:
from tqdm import tqdm

def run_tests(f_bruteforce, f, input_generator):
    """
    Runs random tests using bruteforce solution.

    ## Parameters

    `f_bruteforce`: bruteforce solution.

    `f`: Efficient solution.

    `path`: Path to the input files folder.
    """

    if test_env:
        path = 'tests/tmp.txt'

        failed_tests = 0
        print('Testing in progress...')
        for test_id in tqdm(range(1, 1000+1)):
            input_generator(path)
            expected = f_bruteforce(path)
            actual = f(path)
            
            if expected != actual:
                failed_tests += 1
                print('Test {} failed. Expected {}, actual {}'.format(test_id, expected, actual))

            if failed_tests >= 10:
                print('Testing is stopped after 10 failed tests.')
                break

        if failed_tests == 0:
            print('OK! All tests passed.')
    else:
        print('Tests are skipped. Set test_env variable to True in order to test.')

## Задача 1

Имеется набор данных, состоящий из целых чисел. Необходимо выбрать из каждой пары ровно одно число так, чтобы сумма всех выбранных чисел не делилась на 3 и при этом была минимально возможной. Гарантируется, что искомую сумму получить можно. Программа должна напечатать одно число – минимально возможную сумму, соответствующую условиям задачи.

**Входные данные**: Даны два входных файла: файл A (`27-1a.txt`) и файл B (`27-1b.txt`), каждый из которых содержит в первой строке количество чисел $N (1 ≤ N ≤ 100000)$. Каждая из следующих N строк содержит два натуральных числа, не превышающих по модулю 10000.

**Пример входного файла:**

```
6
1 3
5 12
6 9
5 4
3 3
1 1
```

Для указанных входных данных значением искомой суммы должно быть число 20.


В ответе укажите два числа: сначала искомое значение для файла А, затем для файла B.

In [5]:
from itertools import product

def problem1_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for _ in range(n):
            pair = list(map(int, f.readline().split()))
            a.append(pair)

        result = None

        # option - кортеж вида (i, x), где i - номер пары, x - 0 или 1, индекс элемента пары, который выбран в сумму
        for option in list(product([0, 1], repeat=n)):
            s = 0
            for i, x in enumerate(option):
                s += a[i][x]

            if s % 3 != 0 and (result == None or s < result):
                result = s

        return result

def problem1(path):
    with open(path) as f:
        n = int(f.readline())

        # Очевидно, наша цель - брать все время меньшее число.

        r1 = 100001 # минимальное из всех не взятых в сумму чисел, дающее в остатке от деление на 3 число 1
        r2 = 100001 # минимальное из всех не взятых чисел, дающее в остатке от деление на 3 число 2

        s = 0 # ответ на задачу

        for _ in range(n):
            x, y = map(int, f.readline().split())
            if x > y:
                x, y = y, x

            # теперь x - меньшее из чисел, y - большее

            s += x

            d = y - x
            if d < r1 and d % 3 == 1:
                r1 = d
            if d < r2 and d % 3 == 2:
                r2 = d

        if s % 3 != 0:
            return s
        
        return min(s + r1, s + r2)


solutions[1] = '{} {}'.format(problem1('./data/27data/1/27-1a.txt'), problem1('./data/27data/1/27-1b.txt'))
solutions[1]

'67303 200157496'

In [6]:
def problem1_polyakov(path):
    with open(path) as f:
        n = int(f.readline())
        D = 3

        s, dMin  = 0, 10001
        for i in range(n):
            a, b = map( int, f.readline().split() )
            s += min( a, b )
            d = abs( a-b )
            if d % D > 0:
                dMin = min( d, dMin )

        if s % D != 0:
            return s
        else:
            return s + dMin

In [7]:
from random import randint

def input_generator1(path):
    with open(path, 'w+') as f:
        n = 12
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 30000)) + ' ' + str(randint(1, 30000)) + '\n')

In [8]:
run_tests(problem1_bruteforce, problem1, input_generator1)

Testing in progress...


100%|██████████| 1000/1000 [00:09<00:00, 110.10it/s]

OK! All tests passed.


In [9]:
run_tests(problem1_bruteforce, problem1_polyakov, input_generator1)

Testing in progress...


 54%|█████▍    | 539/1000 [00:04<00:04, 113.17it/s]

Test 517 failed. Expected 128642, actual 127274


100%|██████████| 1000/1000 [00:08<00:00, 111.58it/s]


## Задача 2

In [10]:
def problem2_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for _ in range(n):
            pair = list(map(int, f.readline().split()))
            a.append(pair)

        result = None

        for option in list(product([0, 1], repeat=n)):
            s = 0
            for i, x in enumerate(option):
                s += a[i][x]

            if s % 3 == 0 and (result == None or s > result):
                result = s

        return result

def problem2(path):
    with open(path) as f:
        n = int(f.readline())

        # Очевидно, наша цель - брать все время наибольшее число из пары.

        r11 = 100001
        r12 = 100001
        r21 = 100001
        r22 = 100001

        s = 0 # ответ на задачу

        for _ in range(n):
            x, y = map(int, f.readline().split())
            if x > y:
                x, y = y, x

            # теперь x - меньшее из чисел, y - большее

            s += y

            d = y - x
            
            if d % 3 == 1:
                if d < r11:
                    r12 = r11
                    r11 = d
                elif (d < r12):
                    r12 = d
            
            elif d % 3 == 2:
                if d < r21:
                    r22 = r21
                    r21 = d
                elif d < r22:
                    r22 = d

        if s % 3 == 0:
            return s
        elif s % 3 == 1:
            return max(s - r11, s - r21 - r22)
        else:
            return max(s - r21, s - r11 - r12)


solutions[2] = '{} {}'.format(problem2('./data/27data/2/27-2a.txt'), problem2('./data/27data/2/27-2b.txt'))
solutions[2]

'109737 401536407'

In [11]:
from random import randint

def input_generator2(path):
    with open(path, 'w+') as f:
        n = 12
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 30000)) + ' ' + str(randint(1, 30000)) + '\n')

In [12]:
run_tests(problem2_bruteforce, problem2, input_generator2)

Testing in progress...


100%|██████████| 1000/1000 [00:09<00:00, 108.86it/s]

OK! All tests passed.


## Задача 4

In [13]:
def problem4_incorrect(path):
    with open(path) as f:
        n = int(f.readline())

        s = 0 # ответ на задачу
        r = [10001] * 5 # r[i] - наименьшая разница между x и y по всем парам среди тех разниц, у которых остаток от деления на 5 равен i

        for _ in range(n):
            x, y = map(int, f.readline().split())
            if x > y:
                x, y = y, x

            s += x
            d = y - x
            r[d % 5] = min(r[d % 5], d)

        print(r)

        if s % 5 == 0:
            return s

        return s - r[1]

In [14]:
def problem4_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for _ in range(n):
            pair = list(map(int, f.readline().split()))
            a.append(pair)

        result = None

        for option in list(product([0, 1], repeat=n)):
            s = 0
            for i, x in enumerate(option):
                s += a[i][x]

            if s % 5 == 0 and (result == None or s > result):
                result = s

        return result

def problem4(path):
    with open(path) as f:
        n = int(f.readline())

        s = 0
        m = [10 ** 20] * 5
        for i in range(n):
            x, y = map(int, f.readline().split())
            a = abs(x-y)
            s += max(x,y)

            m[a % 5] = min(m[a % 5], a)

        
        if s % 5 == 1:
            s -= m[1]
        if s % 5 == 2:
            s -= m[2]
        if s % 5 == 3:
            s -= min(m[1]+m[2],m[3])
        if s % 5 == 4:
            s -= min(m[1]+m[3],m[4])

        return s

solutions[4] = '{} {}'.format(problem4('./data/27data/4/27-4a.txt'), problem4('./data/27data/4/27-4b.txt'))
solutions[4]

'123720 402332230'

In [15]:
from random import randint

def input_generator4(path):
    with open(path, 'w+') as f:
        n = 12
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 100000)) + ' ' + str(randint(1, 100000)) + '\n')

In [16]:
run_tests(problem4_bruteforce, problem4, input_generator4)

Testing in progress...


  1%|          | 10/1000 [00:00<00:10, 95.70it/s]

Test 3 failed. Expected 886680, actual 856975
Test 5 failed. Expected 717855, actual 710570
Test 8 failed. Expected 709325, actual 703585
Test 14 failed. Expected 777670, actual 768895
Test 18 failed. Expected 912620, actual 905290
Test 20 failed. Expected 812450, actual 793180


  2%|▏         | 21/1000 [00:00<00:09, 101.04it/s]

Test 22 failed. Expected 680395, actual -99999999999999280624


  3%|▎         | 33/1000 [00:00<00:09, 100.15it/s]

Test 32 failed. Expected 905310, actual 887275
Test 33 failed. Expected 724675, actual 706810
Test 34 failed. Expected 615290, actual 612980
Testing is stopped after 10 failed tests.


## Задача 5

In [17]:
def problem5_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for _ in range(n):
            pair = list(map(int, f.readline().split()))
            a.append(pair)

        result = None

        for option in list(product([0, 1], repeat=n)):
            s = 0
            for i, x in enumerate(option):
                s += a[i][x]

            if s % 5 == 0 and (result == None or s < result):
                result = s

        return result


def problem5(path):
    with open(path) as f:
        n = int(f.readline())
        s = 0

        infty = 10 ** 20
        m = [infty] * 5

        for i in range(n):
            x, y = map(int, f.readline().split())
            a = abs(x - y)
            s += min(x, y)
            m[a % 5] = min(m[a % 5], a)

        if s % 5 == 1:
            s += min(m[1] + m[3], m[4])
        if s % 5 == 2:
            s += min(m[1] + m[2], m[3])
        if s % 5 == 3:
            s += m[2]
        if s % 5 == 4:
            s += m[1]

        return s


def problem5_polyakov(path):
    with open(path) as f:
        n = int(f.readline())
        s = 0

        D = 5
        s, dMin = 0, [10 ** 20] * D
        for i in range(n):
            a, b = map(int, f.readline().split())
            s += min(a, b)
            d = abs(a-b)
            r = d % D
            if r > 0:
                dMinNew = dMin[:]
                for k in range(1, D):
                    r0 = (r + k) % D
                    dMinNew[r0] = min(d+dMin[k], dMinNew[r0])
                dMinNew[r] = min(d, dMinNew[r])
                dMin = dMinNew[:]

        if s % D == 0:
            return s
        else:
            return s + dMin[D - s % D]


In [18]:
problem5_bruteforce('./data/27data/5/27-5a.txt')

75960

In [19]:
solutions[5] = '{} {}'.format(problem5('./data/27data/5/27-5a.txt'), problem5('./data/27data/5/27-5b.txt'))
solutions[5]

'75960 203343860'

In [20]:
from random import randint

def input_generator5(path):
    with open(path, 'w+') as f:
        n = 12
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 30000)) + ' ' + str(randint(1, 30000)) + '\n')

In [21]:
run_tests(problem5_bruteforce, problem5, input_generator5)

Testing in progress...


  1%|          | 11/1000 [00:00<00:09, 106.80it/s]

Test 3 failed. Expected 117435, actual 125880
Test 11 failed. Expected 113345, actual 114945
Test 20 failed. Expected 152625, actual 157210


  2%|▏         | 22/1000 [00:00<00:09, 107.41it/s]

Test 24 failed. Expected 114510, actual 115550


  4%|▍         | 44/1000 [00:00<00:08, 106.96it/s]

Test 26 failed. Expected 118435, actual 121730
Test 31 failed. Expected 159100, actual 159545
Test 32 failed. Expected 139010, actual 100000000000000135094
Test 42 failed. Expected 104465, actual 124515
Test 43 failed. Expected 108300, actual 108810


  5%|▍         | 48/1000 [00:00<00:09, 104.69it/s]

Test 49 failed. Expected 136200, actual 136930
Testing is stopped after 10 failed tests.


In [22]:
run_tests(problem5_bruteforce, problem5_polyakov, input_generator5)

Testing in progress...


100%|██████████| 1000/1000 [00:09<00:00, 106.86it/s]

OK! All tests passed.


## Задача 6

Имеется набор данных, состоящий из положительных целых чисел, каждое из которых не превышает 1000. Требуется найти для этой последовательности контрольное значение – наибольшее число $R$, удовлетворяющее следующим условиям:

* $R$ – произведение двух различных переданных элементов последовательности («различные» означает, что не рассматриваются квадраты переданных чисел, произведения различных, но равных по величине элементов допускаются);
* $R$ делится на 6.

**Входные данные**: Даны два входных файла: файл A (`27-1a.txt`) и файл B (`27-1b.txt`), каждый из которых содержит в первой строке количество чисел $N (1 ≤ N ≤ 100000)$. Каждая из следующих N строк содержит два натуральных числа, не превышающих по модулю 10000.

**Пример входного файла:**

```
6
60
17
3
7
9
60
```

Для указанных входных данных искомое контрольное значение равно 3600.

В ответе укажите два числа: сначала искомое значение для файла А, затем для файла B.

In [23]:
def problem6_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))
        
        result = 0
        for i in range(n):
            for j in range(i+1, n):
                x = a[i] * a[j]
                if x % 6 == 0 and x > result:
                    result = x

        return result

In [24]:
def problem6(path: str):
    with open(path) as f:
        n = int(f.readline())

        m = 0
        m2 = 0
        m3 = 0
        m61 = 0
        m62 = 0

        for i in range(n):
            x = int(f.readline())
            if x % 6 == 0:
                if x > m61:
                    m62 = m61
                    m61 = x
                elif x > m62:
                    m62 = x
            elif x % 3 == 0:
                m3 = max(m3, x)
            elif x % 2 == 0:
                m2 = max(m2, x)
            else:
                m = max(m, x)

        return max(m2 * m3, max(m2, m3, m) * m61, m61 * m62)

In [25]:
def problem6_polyakov(path: str):
    with open(path) as f:
        n = int(f.readline())
        max6 = 0
        maxX = 0
        max2 = 0
        max3 = 0
        for i in range(n):
            x = int(f.readline())
            if x % 6 == 0 and x > max6:
                if max6 > maxX: maxX = max6
                max6 = x
            else:
                if x > maxX: maxX = x
                if x % 2 == 0 and x > max2: max2 = x
                elif x % 3 == 0 and x > max3: max3 = x

        R = max( max6*maxX, max2*max3 )

        return R

In [26]:
solutions[6] = '{} {}'.format(problem6_bruteforce('./data/27data/6/27-6a.txt'), problem6('./data/27data/6/27-6b.txt'))
solutions[6]

'782040 997002'

### Генерация тестов

In [27]:
from random import randint

def input_generator6(path):
    with open(path, 'w+') as f:
        n = 100
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 30000)) + '\n')

In [28]:
run_tests(problem6_bruteforce, problem6, input_generator6)

Testing in progress...


100%|██████████| 1000/1000 [00:05<00:00, 191.46it/s]

OK! All tests passed.


In [29]:
run_tests(problem6_bruteforce, problem6_polyakov, input_generator6)

Testing in progress...


100%|██████████| 1000/1000 [00:05<00:00, 186.98it/s]

OK! All tests passed.


## Задача 7

In [30]:
def problem7_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))
        
        result = 0
        for i in range(n):
            for j in range(i+1, n):
                x = a[i] * a[j]
                if x % 7 == 0 and x % 49 != 0 and x > result:
                    result = x

        return result

In [31]:
def problem7(path: str):
    with open(path) as f:
        n = int(f.readline())
        m7 = 0
        m = 0

        for i in range(n):
            x = int(f.readline())
            if x % 7 == 0 and x % 49 != 0 and x > m7:
                m7 = x
            elif x % 7 != 0 and x > m:
                m = x

        return m7 * m

In [32]:
solutions[7] = '{} {}'.format(problem7_bruteforce('./data/27data/7/27-7a.txt'), problem7('./data/27data/7/27-7b.txt'))
solutions[7]

'692286 952567'

In [33]:
from random import randint

def input_generator7(path):
    with open(path, 'w+') as f:
        n = 100
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 100000)) + '\n')

In [34]:
run_tests(problem7_bruteforce, problem7, input_generator7)

Testing in progress...


100%|██████████| 1000/1000 [00:05<00:00, 199.94it/s]

OK! All tests passed.


## Задача 8

Имеется набор данных, состоящий из положительных целых чисел, каждое из которых не превышает 1000. Они представляют собой результаты измерений, выполняемых прибором с интервалом 1 минута. Требуется найти для этой последовательности контрольное значение – наименьшую сумму квадратов двух результатов измерений, выполненных с интервалом не менее, чем в 5 минут.

In [35]:
def problem8_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))
        
        result = 10 ** 20
        for i in range(n):
            for j in range(i+5, n):
                x = (a[i] ** 2) + (a[j] ** 2)
                if x < result:
                    result = x

        return result

def problem8(path: str):
    with open(path) as f:
        n = int(f.readline())
        window = [0] * 5
        answer = 10 ** 20
        m = None

        for i in range(n):
            if i >= 5 and (m == None or window[i % 5] < m):
                m = window[i % 5]
            x = int(f.readline())
            window[i % 5] = x

            if m != None:
                answer = min(answer, m ** 2 + window[i % 5] ** 2)

        return answer

solutions[8] = '{} {}'.format(problem8('./data/27data/8/27-8a.txt'), problem8('./data/27data/8/27-8b.txt'))
solutions[8]

'11009 200'

In [36]:
def problem8(path: str):
    with open(path) as f:
        n = int(f.readline())
        # window = [0] * 5
        answer = 10 ** 20
        m = 10 ** 20

        for i in range(n):
            x = int(f.readline())

            if i >= 5:
                answer = min(answer, m ** 2 + x ** 2)

            m = min(m, x)

        return answer

solutions[8] = '{} {}'.format(problem8('./data/27data/8/27-8a.txt'), problem8('./data/27data/8/27-8b.txt'))
solutions[8]

'3809 200'

0, 1, 2, 3, 4

1, 2, 3, 4, 5

2, 3, 4, 5, 6

3, 4, 5, 6, 7

5, 6, 7, 8, 4

In [37]:
answers[8]

'11009 200'

In [38]:
problem8_bruteforce('./data/27data/8/27-8a.txt')

11009

In [39]:
problem8('./data/27data/8/27-8a.txt')

3809

In [40]:
from random import randint

def input_generator8(path):
    with open(path, 'w+') as f:
        n = 100
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 100000)) + '\n')

In [41]:
run_tests(problem8_bruteforce, problem8, input_generator8)

Testing in progress...


  2%|▏         | 16/1000 [00:00<00:06, 152.38it/s]

Test 22 failed. Expected 21474434, actual 7303409
Test 29 failed. Expected 13079560, actual 10528132


  5%|▍         | 48/1000 [00:00<00:06, 153.16it/s]

Test 30 failed. Expected 366049, actual 331369
Test 39 failed. Expected 13541098, actual 11150505
Test 51 failed. Expected 34190900, actual 17872180
Test 56 failed. Expected 2962714, actual 1915178


  7%|▋         | 72/1000 [00:00<00:06, 146.94it/s]

Test 62 failed. Expected 1543505, actual 1090949
Test 63 failed. Expected 62215444, actual 49357773
Test 70 failed. Expected 1216034, actual 985538
Test 73 failed. Expected 14861825, actual 3590129
Testing is stopped after 10 failed tests.


## Задача 9

In [42]:
def problem9_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))
        
        result = None
        for i in range(n-6):
            for j in range(i+6, n):
                x = a[i] * a[j]
                if x % 2 == 1 and (result == None or x < result):
                    result = x

        if result == None:
            return -1
        
        return result

In [43]:
problem9_bruteforce('./data/27data/9/27-9a.txt')

2465

## Задача 12

In [44]:
def problem12_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))

        answer = 0
        for i in range(n):
            for j in range(i+1, n):
                if (a[i] * a[j]) % 6 == 0:
                    answer += 1

        return answer

def problem12(path: str):
    with open(path) as f:
        n = int(f.readline())

        answer = 0
        k6 = 0 # количество кратных 6
        k3 = 0 # количество не кратных 6, кратных 3
        k2 = 0 # количество не кратных 6, кратных 2

        for i in range(n):
            x = int(f.readline())
            if x % 6 == 0:
                k6 += 1
            elif x % 3 == 0:
                k3 += 1
            elif x % 2 == 0:
                k2 += 1
        
        answer = k2 * k3 + k6 * (n - k6) + (k6 * (k6-1) // 2)
        return answer

solutions[12] = '{} {}'.format(problem12_bruteforce('./data/27data/12/27-12a.txt'), problem12('./data/27data/12/27-12b.txt'))
solutions[12]

'47 745460178'

## Задача 13

In [45]:
def problem13_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))
        
        answer = 0
        for i in range(n):
            for j in range(i+7, n):
                if (a[i] * a[j]) % 14 == 0:
                    answer += 1

        return answer

def problem13(path: str):
    with open(path) as f:
        n = int(f.readline())
        buf = [int(f.readline()) for i in range(7)]
        answer = 0
        c14 = 0 # количество чисел, кратных 14, с индексами от 0 до i-7
        c2 = 0 # количество чисел, кратных 2, с индексами от 0 до i-7
        c7 = 0 # количество чисел, кратных 7, с индексами от 0 до i-7
        c = 0 # количество чисел, не кратных 2 и 7, с индексами от 0 до i-7

        for i in range(7, n):
            x = int(f.readline())
            if buf[i % 7] % 14 == 0:
                c14 += 1
            elif buf[i % 7] % 2 == 0:
                c2 += 1
            elif buf[i % 7] % 7 == 0:
                c7 += 1
            else:
                c += 1

            buf[i % 7] = x

            if x % 14 == 0:
                answer += c14 + c7 + c2 + c
            elif x % 2 == 0:
                answer += c14 + c7
            elif x % 7 == 0:
                answer += c14 + c2
            else:
                answer += c14
        
        return answer

solutions[13] = '{} {}'.format(problem13_bruteforce('./data/27data/13/27-13a.txt'), problem13('./data/27data/13/27-13b.txt'))
solutions[13]

'30 360137507'

In [46]:
answers[13]

'30 360137507'

In [47]:
from random import randint

def input_generator13(path):
    with open(path, 'w+') as f:
        n = 100
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 100000)) + '\n')

In [48]:
run_tests(problem13_bruteforce, problem13, input_generator13)

Testing in progress...


100%|██████████| 1000/1000 [00:04<00:00, 202.96it/s]

OK! All tests passed.


In [49]:
answers[13]

'30 360137507'

## Задача 14

In [50]:
def problem14_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))

        answer = 0
        for i in range(n):
            for j in range(i+1, n):
                if (a[i] + a[j]) % 12 == 0:
                    answer += 1

        return answer

def problem14(path: str):
    with open(path) as f:
        n = int(f.readline())

        answer = 0
        r = [0] * 12
        for i in range(n):
            x = int(f.readline())
            r[x % 12] += 1

        answer += r[0] * (r[0] - 1) // 2
        answer += r[6] * (r[6] - 1) // 2

        for i in range(1, 6):
            answer += r[i] * r[12-i]
        return answer

solutions[14] = '{} {}'.format(problem14_bruteforce('./data/27data/14/27-14a.txt'), problem14('./data/27data/14/27-14b.txt'))
solutions[14]

'17 150016535'

In [51]:
answers[14]

'17 150016535'

In [52]:
problem14_bruteforce('./data/27data/14/27-14a.txt')

17

In [53]:
problem14('./data/27data/14/27-14a.txt')

17

In [54]:
from random import randint

def input_generator14(path):
    with open(path, 'w+') as f:
        n = 100
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 1000)) + '\n')

In [55]:
run_tests(problem14_bruteforce, problem14, input_generator14)

Testing in progress...


100%|██████████| 1000/1000 [00:04<00:00, 218.48it/s]

OK! All tests passed.


## Задача 15

In [56]:
def problem15_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))
        
        answer = 0
        for i in range(n):
            for j in range(i+5, n):
                s = a[i] + a[j]
                if s % 14 == 0:
                    answer += 1

        return answer

def problem15(path: str):
    with open(path) as f:
        n = int(f.readline())
        answer = 0

        window = [0] * 5

        for i in range(n):
            x = int(f.readline())

        return answer

In [57]:
problem15_bruteforce('./data/27data/15/27-15a.txt')

8

In [58]:
# run_tests(problem15_bruteforce, problem15, input_generator15)

## Задача 18

In [59]:
def problem18_bruteforce(path: str):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))

        answer = 0
        for i in range(n):
            for j in range(i+1, min(i+5, n)):
                if (a[i] * a[j]) % 13 == 0 and (a[i] + a[j]) % 2 == 1:
                    answer += 1

        return answer

In [60]:
answers[18]

'11 17813'

In [61]:
def problem18(path: str):
    with open(path) as f:
        n = int(f.readline())

        return 0

In [62]:
solutions[18] = '{} {}'.format(problem18_bruteforce('./data/27data/18/27-18a.txt'), problem18_bruteforce('./data/27data/18/27-18b.txt'))
solutions[18]

'11 17813'

## №19

Имеется набор данных, состоящий из целых чисел. Необходимо определить максимальное произведение подпоследовательности, состоящей из одного или более идущих подряд элементов.


**Входные данные**: Даны два входных файла: файл A (`27-19a.txt`) и файл B (`27-19b.txt`), каждый из которых содержит в первой строке количество чисел N (1 ≤ N ≤ 100000). Каждая из следующих N строк содержит целое число, не превышающее по модулю 100.

**Пример входного файла:**

```
7
2
3
-2
-3
-1
4
6
```

Для указанных входных данных наибольшее произведение равно 72. Его можно получить для последовательности -3 -1 4 6. 


В ответе укажите два числа: сначала искомое значение для файла А, затем для файла B.



In [63]:
def problem19_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))

        result = float('-inf')
        for length in range(n):
            for l in range(n-length):
                p = 1
                for x in a[l:l+length]:
                    p *= x

                if p > result:
                    result = p

        return result

In [64]:
def problem19(path):
    with open(path) as f:
        ans = 1
        p = 1

        n = int(f.readline())
        for i in range(n):
            x = int(f.readline())
            if x > 0:
                if p > 0:
                    p *= x
                else:
                    if p > ans:
                        ans = p
                    p = 1
            else:
                if p < 0:
                    p *= x
                else:
                    if p > ans:
                        ans = p
                    p = 1
        
        return ans

solutions[19] = '{} {}'.format(problem19_bruteforce('./data/27data/19/27-19a.txt'), problem19('./data/27data/19/27-19b.txt'))
solutions[19]

'663497670 999398400'

In [65]:
def problem19_polyakov(path):
    with open(path) as f:
        n = int(f.readline())

        x = int(f.readline())  # первое число
        dp_min = dp_max = answer = x

        for _ in range(n-1):
            x = int(f.readline())
            t = min(dp_min, dp_max, x)
            dp_max = max(dp_min*x, dp_max*x, x)
            dp_min = t
            answer = max(answer, dp_max)

        return answer

solutions[19] = '{} {}'.format(problem19_bruteforce('./data/27data/19/27-19a.txt'), problem19_polyakov('./data/27data/19/27-19b.txt'))
solutions[19]

'663497670 97618752000'

In [66]:
from random import randint

def input_generator19(path):
    with open(path, 'w+') as f:
        n = 50
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(-100, 100)) + '\n')

In [67]:
run_tests(problem19_bruteforce, problem19, input_generator19)

Testing in progress...


  1%|          | 9/1000 [00:00<00:06, 152.56it/s]

Test 1 failed. Expected 215473683799361521868049896081919360385187209347443802702348288000000000000, actual 95351256
Test 2 failed. Expected 40229477544742467444216881431748971815302001793208432655332526604324700160000000, actual 204120
Test 3 failed. Expected 92713380581601892361734446993699810728103445572111313664983438668595200000000000, actual 307108454400000
Test 4 failed. Expected 1596209203283015522800640734396704000000, actual 13911590
Test 5 failed. Expected 685740214786637345951792267261741090792106731182521735401648947200000000000000, actual 14346300
Test 6 failed. Expected 6780865248247767093718507648590831625639136065115135581320176405053440000000000, actual 26643890
Test 7 failed. Expected 85652731759425283144239048504130331481288033618245891078945243136000000000000, actual 251358030
Test 8 failed. Expected 1028666270504928305761531738224492167626752000000000000, actual 1293576240
Test 9 failed. Expected 12001702691606477028207042001771129158275461888961540323751126630400

## Задача 28

Имеется набор данных, состоящий из пар положительных целых чисел. Необходимо выбрать из каждой пары ровно одно число так, чтобы восьмеричная запись суммы всех выбранных чисел НЕ оканчивалась на 2 и при этом была минимально возможной. Гарантируется, что искомую сумму получить можно. Программа должна напечатать одно число – минимально возможную сумму, соответствующую условиям задачи.

In [68]:
from itertools import combinations, product

def problem28_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for _ in range(n):
            pair = list(map(int, f.readline().split()))
            a.append(pair)

        result = None

        # option - кортеж вида (i, x), где i - номер пары, x - 0 или 1, индекс элемента пары, который выбран в сумму
        for option in list(product([0, 1], repeat=n)):
            s = 0
            for i, x in enumerate(option):
                s += a[i][x]

            if (s % 8) != 2 and (result == None or s < result):
                result = s

        return result

def problem28(path):
    with open(path) as f:
        n = int(f.readline())
        s = 0
        infty = 10 ** 20
        m = infty

        for _ in range(n):
            x, y = map(int, f.readline().split())
            if x > y:
                x, y = y, x
            
            s += x
            d = y - x
            
            if d % 8 != 0:
                m = min(m, d)

        if (s % 8) != 2:
            return s
    
        return s + m

In [69]:
problem28_bruteforce('./data/27data/28/27-28a.txt')

6756

In [70]:
from random import randint

def input_generator28(path):
    with open(path, 'w+') as f:
        n = 10
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 100000)) + ' ' + str(randint(1, 100000)) + '\n')

In [71]:
run_tests(problem28_bruteforce, problem28, input_generator28)

Testing in progress...


100%|██████████| 1000/1000 [00:05<00:00, 197.94it/s]

OK! All tests passed.


In [72]:
solutions[28] = '{} {}'.format(problem28('./data/27data/28/27-28a.txt'), problem28('./data/27data/28/27-28b.txt'))
solutions[28]

'6756 19751921'

## 32

In [73]:
from itertools import combinations, product

def problem32_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for _ in range(n):
            triplet = list(map(int, f.readline().split()))
            a.append(triplet)

        result = None
        
        for option in list(product([0, 1, 2], repeat=n)):
            s = 0
            for i, x in enumerate(option):
                s += a[i][x]

            if s % 11 == 0 and (result == None or s < result):
                result = s

        return result

def problem32(path):
    with open(path) as f:
        n = int(f.readline())
        s = 0

        m = [10 ** 20] * 11

        for _ in range(n):
            triplet = list(map(int, f.readline().split()))
            triplet.sort()

            s += triplet[0]
            m_new = m.copy()

            for i in [1, 2]:
                d = triplet[i] - triplet[0]
                r = d % 11
                if r > 0:
                    for k in range(1, 11):
                        r0 = (r + k) % 11
                        m_new[r0] = min(m[k] + d, m_new[r0])
                    m_new[r] = min(m_new[r], d)
            m = m_new.copy()

        if s % 11 == 0:
            return s

        return s + m[11 - s % 11]

In [74]:
solutions[32] = '{} {}'.format(problem32('./data/27data/32/27-32a.txt'), problem32('./data/27data/32/27-32b.txt'))
solutions[32]

'5896 14078757'

In [75]:
from random import randint

def input_generator32(path):
    with open(path, 'w+') as f:
        n = 10
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 100000)) + ' ' + str(randint(1, 100000)) + ' ' + str(randint(1, 100000)) + '\n')

In [76]:
run_tests(problem32_bruteforce, problem32, input_generator32)

Testing in progress...


100%|██████████| 1000/1000 [01:03<00:00, 15.68it/s]

OK! All tests passed.


In [77]:
answers[32]

'5896 14078757'

## 34

In [78]:
from itertools import combinations, product

def problem34_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for _ in range(n):
            triplet = list(map(int, f.readline().split()))
            a.append(triplet)

        result = None
        
        index_combinations = list(combinations(range(3), 2))
        for option in list(product(index_combinations, repeat=n)):
            s = 0
            for i, pair in enumerate(option):
                s += a[i][pair[0]] + a[i][pair[1]]

            if s % 6 == 0 and (result == None or s > result):
                result = s

        return result

def problem34(path):
    with open(path) as f:
        n = int(f.readline())
        s = 0

        m = 10 ** 20

        for _ in range(n):
            a = list(map(int, f.readline().split()))
            a.sort()

            s += a[0] + a[1]

            r = a[2] - a[1]

            if r < m:
                if r % 6 == 0:
                    m = r
                elif a[2] - a[0] and (a[2] - a[0]) % 6 == 0:
                    m = a[2] - a[0]

        if s % 6 == 0:
            return s

        return s - r

In [79]:
from random import randint

def input_generator34(path):
    with open(path, 'w+') as f:
        n = 10
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 100000)) + ' ' + str(randint(1, 100000)) + ' ' + str(randint(1, 100000)) + '\n')

In [141]:
run_tests(problem34_bruteforce, problem34, input_generator34)

Testing in progress...


  0%|          | 1/1000 [00:00<01:49,  9.09it/s]

Test 1 failed. Expected 1157178, actual 705804


  0%|          | 3/1000 [00:00<01:47,  9.29it/s]

Test 2 failed. Expected 1271952, actual 745542
Test 3 failed. Expected 1304742, actual 644648


...

Test 6 failed. Expected 1258944, actual 726462
Test 7 failed. Expected 1123812, actual 686655


  1%|          | 9/1000 [00:00<01:46,  9.27it/s]

Test 8 failed. Expected 1107348, actual 588264
Test 9 failed. Expected 1286160, actual 828747


  1%|          | 9/1000 [00:01<01:58,  8.36it/s]

Test 10 failed. Expected 1225302, actual 854580
Testing is stopped after 10 failed tests.


In [81]:
solutions[34] = '{} {}'.format(problem34('./data/27data/34/27-34a.txt'), problem34('./data/27data/34/27-34b.txt'))
solutions[34]

'9572 30227092'

In [82]:
answers[34]

'9750 30227250'

## Задача 35

In [135]:
def problem35_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))
        
        answer = 0
        for l in range(n):
            for r in range(l+1, n):
                if a[l] > 0 and a[r] > 0 and (a[l] + a[r]) % 2 == 0 and 0 in a[l:r]:
                    answer += 1

        return answer

def problem35(path):
    with open(path) as f:
        n = int(f.readline())
        z = -1 # позиция ближайшего слева нуля к i-му элементу
        c1 = 0 # количество положительных нечетных элементов до текущего z-го
        c2 = 0 # количество положительных четных элементов до текущего z-го

        r1 = 0
        r2 = 0
        answer = 0

        for i in range(n):
            x = int(f.readline())
            if x == 0:
                z = i
                c1 += r1
                c2 += r2
                r1 = 0
                r2 = 0
            elif x > 0:
                if x % 2 == 0:
                    answer += c2
                    r2 += 1
                else:
                    answer += c1
                    r1 += 1


        return answer


solutions[35] = '{} {}'.format(problem35_bruteforce('./data/27data/35/27-35a.txt'), problem35('./data/27data/35/27-35b.txt'))
solutions[35]

'51 24019058'

In [136]:
answers[35]

'51 24019058'

In [139]:
from random import randint

def input_generator35(path):
    with open(path, 'w+') as f:
        n = 10
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(0, 100000)) + '\n')

In [140]:
run_tests(problem35_bruteforce, problem35, input_generator35)

Testing in progress...


100%|██████████| 1000/1000 [00:04<00:00, 244.87it/s]

OK! All tests passed.


## Задача 74

In [83]:
def problem74_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))
        
        answer = 0
        for l in range(n):
            for r in range(l+1, min(l+21, n)):
                if sum(a[l:r]) % 39 == 0:
                    answer += 1
        
        return answer

def problem74(path):
    with open(path) as f:
        n = int(f.readline())
        answer = 0

        s = [int(f.readline())] # последние 20 префикс сумм
        for i in range(1, 20):
            s.append(s[i-1] + int(f.readline()))

        for k in range(19):
            if s[-1] % 39 == s[k] % 39:
                answer += 1

        for i in range(n-20):
            for k in range(19):
                if s[-1] % 39 == s[k] % 39:
                    answer += 1
            x = int(f.readline())
            s.pop(0)
            s.append(s[-1] + x)
        
        return answer

solutions[74] = '{} {}'.format(problem74_bruteforce('./data/27data/74/27-74a.txt'), problem74_bruteforce('./data/27data/74/27-74b.txt'))
solutions[74]

'21 51023'

In [84]:
answers[74]

'21 51023'

In [85]:
problem74_bruteforce('./data/27data/74/27-74b.txt')

51023

In [86]:
from random import randint

def input_generator74(path):
    with open(path, 'w+') as f:
        n = 100
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 100000)) + '\n')

In [87]:
run_tests(problem74_bruteforce, problem74, input_generator74)

Testing in progress...


  1%|          | 9/1000 [00:00<00:06, 153.86it/s]

Test 1 failed. Expected 36, actual 28
Test 2 failed. Expected 38, actual 30
Test 3 failed. Expected 50, actual 46
Test 4 failed. Expected 52, actual 46
Test 5 failed. Expected 38, actual 30
Test 6 failed. Expected 61, actual 50
Test 7 failed. Expected 48, actual 39
Test 8 failed. Expected 41, actual 38
Test 9 failed. Expected 57, actual 53
Test 10 failed. Expected 51, actual 43
Testing is stopped after 10 failed tests.


In [88]:
problem74_bruteforce('./data/27data/74/27-74a.txt')

21

## Задача 76

In [89]:
def problem76_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))

        answer = 0
        for i in range(n):
            for j in range(i+2, n):
                if (a[i] + a[j]) % 3 == 0 and (sum(a[i+1:j]) % 2 == 0):
                    answer += 1

        return answer

def problem76(path):
    with open(path) as f:
        n = int(f.readline())
        c0 = [0] * 3 # c[j] - количество четных до i-1 итерации, дающих при делении на 3 остаток j
        c1 = [0] * 3 # c[j] - количество нечетных до i-1 итерации, дающих при делении на 3 остаток j

        x_prev = int(f.readline())
        answer = 0
        s = 0

        for i in range(1, n):
            x = int(f.readline())
            s += x_prev
            r = x % 3

            if s % 2 == 0:
                answer += c0[(3-r) % 3]
            else:
                answer += c1[(3-r) % 3]

            if s % 2 == 0:
                c0[x_prev % 3] += 1
            else:
                c1[x_prev % 3] += 1

            x_prev = x

        return answer

In [90]:
solutions[76] = '{} {}'.format(problem76_bruteforce('./data/27data/76/27-76a.txt'), problem76('./data/27data/76/27-76b.txt'))
solutions[76]

'27 833306999'

In [91]:
answers[76]

'27 833306999'

In [92]:
from random import randint

def input_generator76(path):
    with open(path, 'w+') as f:
        n = 20
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 50)) + '\n')

In [93]:
run_tests(problem76_bruteforce, problem76, input_generator76)

Testing in progress...


100%|██████████| 1000/1000 [00:04<00:00, 237.93it/s]

OK! All tests passed.


## 79

In [94]:
from collections import defaultdict

def problem79_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()

        for i in range(n):
            a.append(int(f.readline()))

        answer = 0
        for i in range(n):
            for j in range(i, n):
                if a[i] % 21 == 0 and a[i] == a[j] * a[j]:
                    answer = max(answer, j-i+1)

        return answer

def problem79(path):
    with open(path) as f:
        n = int(f.readline())
        m = defaultdict(lambda: None)

        answer = 0
        for i in range(n):
            x = int(f.readline())
            if x % 21 == 0 and m[x] == None:
                m[x] = i

            if m[x ** 2] != None:
                answer = max(answer, i - m[x ** 2] + 1)

        return answer

In [95]:
from random import randint

def input_generator79(path):
    with open(path, 'w+') as f:
        n = 20
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 50000)) + '\n')

In [96]:
run_tests(problem79_bruteforce, problem79, input_generator79)

Testing in progress...


100%|██████████| 1000/1000 [00:04<00:00, 245.70it/s]

OK! All tests passed.


In [97]:
answers[79]

'337 148613'

In [98]:
problem79_bruteforce('./data/27data/79/27-79a.txt')

337

In [99]:
problem79('./data/27data/79/27-79b.txt')

148613

## 82

In [100]:
def isPrime(x: int):
    if x == 1:
        return False

    i = 2
    while i * i <= x:
        if x % i == 0:
            return False
        i += 1
    
    return True

def problem82_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        k = 9
        a = list()
        for i in range(n):
            a.append(int(f.readline()))

        result = 0
        for length in range(1, n+1):
            for l in range(n-length+1):
                c = 0
                s = sum(a[l:l+length])
                for i in range(l, l+length):
                    # s += a[i]
                    if isPrime(a[i]):
                        c += 1

                if c % k == 0 and s > result:
                    result = s

        return result

def problem82(path):
    with open(path) as f:
        n = int(f.readline())
        k = 9
        
        count = 0 # количество простых чисел
        s = 0 # сумма всех чисел (на i-й итерации это i-ая префикс-сумма)
        infty = 10 ** 20
        tailSum = [0] + [infty] * (k-1) # tailSum[j] - наименьшее значение префикс-суммы, в которой количество простых чисел дает остаток j при делении на k
        answer = 0

        for i in range(n):
            x = int(f.readline())
            s += x
            if isPrime(x):
                count += 1
            
            r = count % k
            # if tailSum[r] is None:
            #     tailSum[r] = s # такое заполнение обеспечивает, что tailSum[r] - наименьшая префикс-сумма, ведь префикс-суммы неубывают с увеличением индекса
            # elif count >= k:
            #     answer = max(answer, s - tailSum[r])

            if r == 0 and s > answer: # если r = 0, то подходит сама префикс-сумма
                answer = max(answer, s)
            elif tailSum[r] != infty and s - tailSum[r] > answer:
                answer = s - tailSum[r]


            tailSum[r] = min(tailSum[r], s)

        return answer

In [101]:
problem82_bruteforce('./data/27data/82/27-82a.txt')

67645

In [102]:
problem82('./data/27data/82/27-82a.txt')

67645

In [103]:
solutions[82] = '{} {}'.format(problem82_bruteforce('./data/27data/82/27-82a.txt'), problem82('./data/27data/82/27-82b.txt'))
solutions[82]

'67645 69976583'

In [104]:
from random import randint

def input_generator82(path):
    with open(path, 'w+') as f:
        n = 15
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(1, 10000)) + '\n')

In [105]:
run_tests(problem82_bruteforce, problem82, input_generator82)

Testing in progress...


100%|██████████| 1000/1000 [00:04<00:00, 200.86it/s]

OK! All tests passed.


## 86

In [106]:
def problem86_bruteforce(path):
    with open(path) as f:
        n, k = map(int, f.readline().split())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))

        result = float('-inf')
        for length in range(1, n+1):
            for l in range(n-length+1):
                s = sum(a[l:l+length])
                c = 0
                for x in a[l:l+length]:
                    if x < 0 and abs(x) % 10 == 7:
                        c += 1

                if c % k == 0 and s > result:
                    result = s

        return result

def problem86(path):
    with open(path) as f:
        n, k = map(int, f.readline().split())
        answer = 0
        count = 0 # количество отрицательных, оканчивающихся на 7
        s = 0 # сумма всех чисел

        infty = 10 ** 20
        m = [infty] * k # m[i] - минимальная префикс-сумма, в которой количество отрицательных чисел, оканчивающихся на 7, кратно k

        for _ in range(n):
            x = int(f.readline())
            s += x

            if abs(x) % 10 == 7 and x < 0:
                count += 1

            r = count % k

            if r == 0 and s > answer: # если r = 0, то подходит сама префикс-сумма
                answer = s
            elif m[r] != infty and s - m[r] > answer:
                answer = s - m[r]

            m[r] = min(m[r], s)
        
        return answer

solutions[86] = '{} {}'.format(problem86_bruteforce('./data/27data/86/27-86a.txt'), problem86('./data/27data/86/27-86b.txt'))
solutions[86]

'15218 253665'

In [107]:
from random import randint

def input_generator86(path):
    with open(path, 'w+') as f:
        n = 10
        k = randint(1, 15)
        f.write(str(n) + ' ' + str(k) + '\n')

        for i in range(n):
            f.write(str(randint(1, 10000)) + '\n')

In [108]:
run_tests(problem86_bruteforce, problem86, input_generator86)

Testing in progress...


100%|██████████| 1000/1000 [00:04<00:00, 224.16it/s]

OK! All tests passed.


## 87

In [109]:
def isSpecial(x):
    if x >= 0:
        return False

    x = abs(x)
    
    while x > 0:
        if x % 5 == 2:
            return False
        x //= 5

    return True

def problem87_bruteforce(path):
    with open(path) as f:
        n, k = map(int, f.readline().split())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))

        result = -10001
        # for length in range(1, n+1):
        #     for l in range(n-length+1):
        for l in range(n):
            for r in range(l, n):
                s = sum(a[l:r])
                c = 0
                for x in a[l:r]:
                    if isSpecial(x):
                        c += 1

                if c % k == 0 and s > result:
                    result = s

        return result

def problem87(path):
    with open(path) as f:
        n, k = map(int, f.readline().split())
        answer = 0
        count = 0 # количество особенных
        s = 0 # сумма всех чисел

        infty = 10 ** 20
        m = [infty] * k # m[i] - минимальная префикс-сумма, в которой количество особенных кратно k

        for _ in range(n):
            x = int(f.readline())
            s += x

            if isSpecial(x):
                count += 1

            r = count % k

            if r == 0 and s > answer:
                answer = max(answer, s)
            elif m[r] != infty and s - m[r] > answer:
                answer = s - m[r]

            m[r] = min(m[r], s)
        
        return answer

solutions[87] = '{} {}'.format(problem87_bruteforce('./data/27data/87/27-87a.txt'), problem87('./data/27data/87/27-87b.txt'))
solutions[87]

'15406 256483'

In [110]:
problem87_bruteforce('./data/27data/87/27-87a.txt')

15406

In [111]:
problem87('./data/27data/87/27-87a.txt')

15406

## 88

In [112]:
def isSpecial(x):
    if x >= 0:
        return False

    x = -x

    i = 2
    while i * i <= x:
        if x % i == 0:
            return False
        i += 1

    return True

def problem88_bruteforce(path):
    with open(path) as f:
        n, k = map(int, f.readline().split())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))

        result = -10001
        for l in range(n):
            for r in range(l, n):
                s = sum(a[l:r])
                c = 0
                for x in a[l:r]:
                    if isSpecial(x):
                        c += 1

                if c % k == 0 and s > result:
                    result = s

        return result

def problem88(path):
    with open(path) as f:
        n, k = map(int, f.readline().split())
        answer = 0
        count = 0 # количество особенных
        s = 0 # сумма всех чисел

        infty = 10 ** 20
        m = [infty] * k # m[i] - минимальная префикс-сумма, в которой количество особенных кратно k

        for _ in range(n):
            x = int(f.readline())
            s += x

            if isSpecial(x):
                count += 1

            r = count % k

            if r == 0 and s > answer:
                answer = max(answer, s)
            elif m[r] != infty and s - m[r] > answer:
                answer = s - m[r]

            m[r] = min(m[r], s)

        return answer

solutions[88] = '{} {}'.format(problem88('./data/27data/88/27-88a.txt'), problem88('./data/27data/88/27-88b.txt'))
solutions[88]

'11527919 168873874'

In [113]:
problem88_bruteforce('./data/27data/88/27-88a.txt')

11527919

In [114]:
def special(x):
    if x > 0:
        return False
    if x == 1 or x == 0:
        return False
    x = abs(x)
    l = 2
    while l*l <= x:
        if x % l == 0:
            return False
        l += 1
    return True

def problem88_Artem(path):
    f = open(path)
    n, k = map(int, f.readline().split())

    max_sum = -10**20
    a = [10**20]*k

    sum = 0
    c = 0
    for i in range(n):
        x = int(f.readline())
        sum += x

        if special(x):
            c += 1

        r = c % k
        if r == 0:
            max_sum = max(max_sum, sum)

        max_sum = max(max_sum, sum - a[r])

        a[r] = min(a[r], sum)
    return max_sum

In [115]:
answers[88]

'11527919 168873874'

In [116]:
from random import randint

def input_generator88(path):
    with open(path, 'w+') as f:
        n = 15
        k = randint(1, 6)
        f.write(str(n) + ' ' + str(k) + '\n')

        for i in range(n):
            f.write(str(randint(-100000, 100000)) + '\n')

In [117]:
run_tests(problem88_bruteforce, problem88_Artem, input_generator88)

Testing in progress...


  2%|▏         | 20/1000 [00:00<00:04, 197.04it/s]

Test 5 failed. Expected 45586, actual 79831
Test 12 failed. Expected 293586, actual 384318
Test 16 failed. Expected 230487, actual 288010
Test 22 failed. Expected 127743, actual 202424
Test 26 failed. Expected 275396, actual 288463
Test 30 failed. Expected 194449, actual 196272


  4%|▍         | 40/1000 [00:00<00:04, 194.23it/s]

Test 39 failed. Expected 170644, actual 211933
Test 40 failed. Expected 126979, actual 130432


  6%|▋         | 63/1000 [00:00<00:04, 197.49it/s]

Test 62 failed. Expected 287040, actual 343865
Test 64 failed. Expected 434038, actual 505579
Testing is stopped after 10 failed tests.


In [118]:
def isSpecNumber(x):
    if x >= 0:
        return False
    x = abs(x)
    if x <= 1:
        return False
    d = 2
    while d*d <= x:
        if x % d == 0:
            return False
        d += 1
    return True

def problem88_polyakov(path):
    f = open(path)
    n, K = map(int, f.readline().split())

    maxSum = -10**20
    minTails = [10**20]*K

    sumTotal = 0
    count = 0
    for i in range(n):

        x = int(f.readline())
        sumTotal += x

        if isSpecNumber(x):
            count += 1

        r = count % K
        if r == 0 and sumTotal > maxSum:
            maxSum = sumTotal

        if minTails[r] != 10**20 and sumTotal - minTails[r] > maxSum:
            maxSum = sumTotal - minTails[r]

        minTails[r] = min(minTails[r], sumTotal)

    f.close()

    return maxSum

In [119]:
run_tests(problem88_bruteforce, problem88_polyakov, input_generator88)

Testing in progress...


  0%|          | 0/1000 [00:00<?, ?it/s]

Test 5 failed. Expected 210998, actual 219473


  2%|▏         | 19/1000 [00:00<00:05, 189.06it/s]

Test 8 failed. Expected 141074, actual 175871
Test 14 failed. Expected 269142, actual 296796
Test 15 failed. Expected 439554, actual 499926
Test 17 failed. Expected 184489, actual 249536
Test 21 failed. Expected 575768, actual 638826
Test 22 failed. Expected 340224, actual 388937
Test 31 failed. Expected 200658, actual 248505


  4%|▍         | 38/1000 [00:00<00:05, 178.57it/s]

Test 38 failed. Expected 405117, actual 490084


  4%|▍         | 42/1000 [00:00<00:05, 175.00it/s]

Test 43 failed. Expected 224104, actual 257470
Testing is stopped after 10 failed tests.


## 91

In [120]:
def isSpecial(x):
    if x >= 0:
        return False

    x = abs(x)
    
    s = 0
    while x > 0:
        s += x % 3
        x //= 3

    return s == 12

def problem91_bruteforce(path):
    with open(path) as f:
        n, k, d = map(int, f.readline().split())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))

        result = -10001
        for length in [x * d for x in range(1, n // d + 1)]:
            for l in range(n-length+1):
                r = l+length
                s = sum(a[l:r])
                c = 0
                for x in a[l:r]:
                    if isSpecial(x):
                        c += 1

                if c % k == 0 and s > result:
                    result = s

        return result

def problem91(path):
    with open(path) as f:
        n, k, d = map(int, f.readline().split())
        answer = -10001
        s = 0
        count = 0

        infty = 10 ** 20
        m = [infty] * k # m[i] - минимальная префикс-сумма, в которой количество особенных кратно k
        c = [0] * k # c[i] - номер итерации, на которой встречена префикс-сумма m[i]

        for i in range(n):
            x = int(f.readline())
            s += x

            if isSpecial(x):
                count += 1

            r = count % k

            if s > answer and r == 0 and i % d == 0:
                answer = s
            elif m[r] != infty and s - m[r] > answer and c[r] % d == i % d:
                answer = s - m[r]
            
            if s < m[r]:
                m[r] = s
                c[r] = i

        return answer

solutions[91] = '{} {}'.format(problem91('./data/27data/91/27-91a.txt'), problem91('./data/27data/91/27-91b.txt'))
solutions[91]

'6597803 168674858'

In [121]:
problem91_bruteforce('./data/27data/91/27-91a.txt')

6597803

In [122]:
problem91('./data/27data/91/27-91a.txt')

6597803

## 92

In [123]:
def isSpecial(x):
    return x > 0 and x % 2 == 0

def problem92_bruteforce(path):
    with open(path) as f:
        n = int(f.readline())
        a = list()
        for i in range(n):
            a.append(int(f.readline()))

        result = -10001
        for length in range(n):
            for l in range(n-length+1):
                r = l+length
                s = 0
                c = 0
                for x in a[l:r]:
                    s += x
                    if isSpecial(x):
                        c += 1

                if c == 1 and s > result:
                    result = s

        return result

def problem92(path):
    with open(path) as f:
        n = int(f.readline())
        answer = 0
        count = 0 # количество особенных
        s = 0 # сумма всех чисел (текущая префикс-сумма на i-й итерации)

        infty = 10 ** 20
        m = infty # минимальная префикс-сумма, в которой count-1 особенных чисел
        m_next = infty # минимальная префикс-сумма, в которой count особенных чисел

        for i in range(n):
            x = int(f.readline())
            s += x

            if isSpecial(x):
                count += 1
                m = m_next
                m_next = infty

            if count == 1 and s > answer:
                answer = max(answer, s)
            elif count > 1:
                answer = max(answer, s - m)
                m_next = min(m_next, s)
        
        return answer

solutions[92] = '{} {}'.format(problem92('./data/27data/92/27-92a.txt'), problem92('./data/27data/92/27-92b.txt'))
solutions[92]

'4710 7368'

In [124]:
def problem92_Artem(path):
    def special(x):
        if x > 0 and x % 2 == 0:
            return True

    f = open(path)
    n = int(f.readline())

    max_sum = -10**20

    sum = 0
    a = [10**20] * 2
    for i in range(n):
        x = int(f.readline())
        sum += x

        if special(x):
            a[0] = a[1]
            a[1] = sum

        max_sum = max(max_sum, sum - a[0])

        a[1] = min(a[1], sum)
    return max_sum

In [125]:
problem92_bruteforce('./data/27data/92/27-92a.txt')

4710

In [126]:
problem92_Artem('./data/27data/92/27-92b.txt')

7368

In [127]:
answers[92]

'4710 7368'

In [128]:
from random import randint

def input_generator92(path):
    with open(path, 'w+') as f:
        n = 15
        f.write(str(n) + '\n')

        for i in range(n):
            f.write(str(randint(-10000, 10000)) + '\n')

In [129]:
run_tests(problem92_bruteforce, problem92, input_generator92)

Testing in progress...


  2%|▏         | 15/1000 [00:00<00:05, 194.81it/s]

Test 3 failed. Expected 16479, actual 12089
Test 4 failed. Expected 10809, actual 10135
Test 7 failed. Expected 14755, actual 11283
Test 8 failed. Expected 14047, actual 12023
Test 10 failed. Expected 8438, actual 0
Test 11 failed. Expected 8100, actual 7576
Test 13 failed. Expected 18084, actual 8932
Test 14 failed. Expected 21012, actual 8747
Test 15 failed. Expected -10001, actual 0
Test 16 failed. Expected 6920, actual 0
Testing is stopped after 10 failed tests.


In [130]:
run_tests(problem92_bruteforce, problem92, input_generator92)

Testing in progress...


  0%|          | 0/1000 [00:00<?, ?it/s]

Test 1 failed. Expected 17044, actual 10942
Test 2 failed. Expected 14766, actual 10357
Test 4 failed. Expected 23456, actual 9592
Test 7 failed. Expected 14063, actual 12793
Test 8 failed. Expected 21390, actual 10747
Test 11 failed. Expected 13021, actual 8111
Test 12 failed. Expected 14043, actual 8382
Test 13 failed. Expected 7128, actual 6255
Test 15 failed. Expected 16780, actual 0


  2%|▏         | 15/1000 [00:00<00:04, 202.71it/s]

Test 16 failed. Expected 10879, actual 9315
Testing is stopped after 10 failed tests.


## Проверка решений

In [131]:
ok_flag = True

solutions = [str(solution) if solution != None else None for solution in solutions]
solved = []

for i in range(1, len(answers)):
    if solutions[i] is not None:
        if solutions[i].split() == answers[i].split():
            solved.append(i)
        else:
            print('Error in problem {}. Correct Answer: {}. Given: {}'.format(i, answers[i], solutions[i]))
            ok_flag = False

if ok_flag:
    print('OK!')
print('Solved problems: {}'.format(solved))

Error in problem 8. Correct Answer: 11009 200. Given: 3809 200
Error in problem 19. Correct Answer: 663497670999806000. Given: 663497670 97618752000
Error in problem 34. Correct Answer: 9750 30227250. Given: 9572 30227092
Solved problems: [1, 2, 4, 5, 6, 7, 12, 13, 14, 18, 28, 32, 74, 76, 82, 86, 87, 88, 91, 92]
